In [1]:
from cobra import io
from cobra.core import Metabolite
from cobra import flux_analysis
from Functions import *
from IPython import display
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

#import sbml file
fname = "C:\\Users\\FFAR\\Downloads\\plantcoremetabolism-model\\PlantCoreMetabolism_v1_3_2.xml"
model = io.read_sbml_model(fname)
display.clear_output()

from sweetlovegroup.transform import fixModelCompatibilityIssueCobra015
model = fixModelCompatibilityIssueCobra015(model,fname)


#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
rxn.add_metabolites({met:1})



model.metabolites.get_by_id("aL_1_PHOSPHATIDYL_GLYCEROL_P_m").remove_from_model()
model.metabolites.get_by_id("aL_1_PHOSPHATIDYL_GLYCEROL_P_p").remove_from_model()

model.reactions.get_by_id("PGPPHOSPHA_RXN_m").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_m"):-0.03,
                                                               model.metabolites.get_by_id("PROTON_m"):-0.03})
model.reactions.get_by_id("PHOSPHAGLYPSYN_RXN_m").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_m"):0.03,
                                                                   model.metabolites.get_by_id("PROTON_m"):0.03})
model.reactions.get_by_id("PGPPHOSPHA_RXN_p").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):-0.03,
                                                               model.metabolites.get_by_id("PROTON_p"):-0.03})
model.reactions.get_by_id("PHOSPHAGLYPSYN_RXN_p").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):0.03,
                                                                   model.metabolites.get_by_id("PROTON_p"):0.03})
model.reactions.get_by_id("LPG_biosynthesis_c").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):-0.03})

In [2]:
import pandas as pd
biomass = pd.DataFrame(data={"":["sSUCROSE_b","GLC_c","FRU_c","Starch_b","Cellulose_b","Xylan_b",
                                 "L_PHOSPHATIDATE_p","PHOSPHATIDYL_CHOLINE_r",
                                 "L_1_PHOSPHATIDYL_ETHANOLAMINE_r","DIACYLGLYCEROL_p",
                                 "Galactosyl_galactosyl_diacyl_glycerols_p",
                                 "D_Galactosyl_12_diacyl_glycerols_p","2_Lysophosphatidylcholines_r",
                                 "Lysophosphatidylglycerols_r","Triacylglycerols_p",
                                 "L_1_PHOSPHATIDYL_GLYCEROL_p","L_1_phosphatidyl_inositols_r",
                                 "SULFOQUINOVOSYLDIACYLGLYCEROL_p","Protein_b",
                                 "sMAL_b","sCIT_b","sFUM_b","ARG_c","HIS_c","LYS_c","sASP_b",
                                 "sGLU_b","sSER_b","THR_c","ASN_c","sGLN_b","CYS_c",
                                 "GLY_c","PRO_c","sALA_b","VAL_c","ILE_c","LEU_c",
                                 "MET_c","PHE_c","TYR_c","TRP_c","sGABA_b","PALMITATE_p",
                                 "CPD_9245_p","CPD_17412_p","CPD_17291_p","STEARIC_ACID_p",
                                 "OLEATE_CPD_p","Octadecadienoate_p","LINOLENIC_ACID_p",
                                 "ARACHIDIC_ACID_p","CPD_16709_p","DOCOSANOATE_p","SUC_c",
                                 "FUM_c","MAL_c","CIS_ACONITATE_c","CIT_c"],
                             "leaf":[0.0]*59,"stem":[0.0]*59,"root":[0.0]*59,"seed":[0.0]*59,},dtype="float64")
biomass = biomass.set_index("")

In [3]:
proteinMW = model.metabolites.get_by_id("Protein_b").formula_weight

cobra\core\metabolite.py:104 UserWarning: 17.2020450391 is not an integer (in formula C17.2020450391H32.8863668319N4.953624517899999O9.218271605099998S0.1420323181)
cobra\core\metabolite.py:104 UserWarning: 32.8863668319 is not an integer (in formula C17.2020450391H32.8863668319N4.953624517899999O9.218271605099998S0.1420323181)
cobra\core\metabolite.py:104 UserWarning: 4.953624517899999 is not an integer (in formula C17.2020450391H32.8863668319N4.953624517899999O9.218271605099998S0.1420323181)
cobra\core\metabolite.py:104 UserWarning: 9.218271605099998 is not an integer (in formula C17.2020450391H32.8863668319N4.953624517899999O9.218271605099998S0.1420323181)
cobra\core\metabolite.py:104 UserWarning: 0.1420323181 is not an integer (in formula C17.2020450391H32.8863668319N4.953624517899999O9.218271605099998S0.1420323181)


### Mature leaf

Extract CO2 assimilation rate for PPFD = 600 umol/m2/s between 11:00 and 13:00h from Liu et al 2004, Figure 3

<img src="references/Screenshot_20190717_132513.png" height=50% width=50%>

In [4]:
df_matureleaf_data1 = pd.DataFrame(data={"DAA":[-10.9638294010889,-9.92869328493648,
                                               -9.02599818511797,-7.9920054446461,
                                               -6.98697822141561,-4.05432849364791,
                                               -3.05117967332123,-2.04432849364791,
                                               -1.02035390199637,2.90665154264973,
                                               3.94565335753176,5.86627949183303,
                                               6.86583484573503,9.80232304900181],
                                        "Photosynthesis (umol/m2/s)":[18.4210526315789,18.2395644283121,
                                                                      14.6460980036298,15.989110707804,
                                                                      15.9528130671506,25.7531760435572,
                                                                      28.2214156079855,25.7531760435572,
                                                                      20.4537205081669,24.4464609800363,
                                                                      19.1107078039927,18.2758620689655,
                                                                      25.535390199637,30.2177858439201]})
df_matureleaf_data1

,DAA,Photosynthesis (umol/m2/s)
0,-10.963829,18.421053
1,-9.928693,18.239564
2,-9.025998,14.646098
3,-7.992005,15.989111
4,-6.986978,15.952813
5,-4.054328,25.753176
6,-3.051180,28.221416
7,-2.044328,25.753176
8,-1.020354,20.453721
9,2.906652,24.446461


Extract CO2 assimilation rate for PPFD = 1000 µmol m−2 s−1 from Figure 1 of Weston et al 2011

<img src="references/Screenshot_20190924_144100.png" height=50% width=50%>

In [5]:
df_matureleaf_data2 = pd.DataFrame(data={"Leaf temperature (C)":[24.9632352941176,28.0882352941176,
                                                                 30.0367647058823,32.1691176470588,
                                                                 35.2941176470588,38.2720588235294,
                                                                 40.0367647058824,41.9852941176471],
                                        "Photosynthesis (umol/m2/s)":[18.9105151864667,22.8457003716519,
                                                                      23.5731449442522,25.3471421248238,
                                                                      25.9489939766756,24.2626233499936,
                                                                      21.5252146610278,17.5467768806869]})
df_matureleaf_data2

,Leaf temperature (C),Photosynthesis (umol/m2/s)
0,24.963235,18.910515
1,28.088235,22.845700
2,30.036765,23.573145
3,32.169118,25.347142
4,35.294118,25.948994
5,38.272059,24.262623
6,40.036765,21.525215
7,41.985294,17.546777


In [6]:
df_matureleaf_data3 = pd.DataFrame(data={"Leaf temperature (C)":[20.1102941176471,24.9264705882353,
                                                                 30.1838235294118,35.1102941176471,
                                                                 39.5588235294118],
                                        "Dark respiration (umol/m2/s)":[0.662433155080194,0.965304303539575,
                                                                      1.58024573465748,2.50658899923604,
                                                                      2.75719378660553]})
df_matureleaf_data3

,Leaf temperature (C),Dark respiration (umol/m2/s)
0,20.110294,0.662433
1,24.926471,0.965304
2,30.183824,1.580246
3,35.110294,2.506589
4,39.558824,2.757194


Extract diurnal starch, sucrose, hemicellulose and simple sugar content from Clements 1930, Figure 6, 9, 13 and 14. Based on Figure 1 and 2, dawn = 8am and dusk = 8pm

<img src="references/Screenshot_20190717_161155.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160304.png" height=50% width=50%>

<img src="references/Screenshot_20190717_161259.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160242.png" height=50% width=50%>

In [7]:
df_matureleaf_content_1 = pd.DataFrame(data={"Series":[1,2,3,4],
                                           "starch_dawn (g/100gFW)":[0.4245676464308603,0.4974291547005305,
                                                                     0.8895379007845832,1.2526663271935523],
                                           "hemicellulose_dawn (g/100gFW)":[1.089463383389083,
                                                                            1.8922736237500428,
                                                                            0.8616451473404596,
                                                                            1.1221330617548415],
                                           "simple sugars_dawn (g/100gFW)":[0.2719414988257076,
                                                                            0.6501251398452184,
                                                                            0.5686189976083096,
                                                                            0.5533809766290336],
                                           "sucrose_dawn (g/100gFW)":[0.2719414988257076,0.454127308166961,
                                                                      0.09770538968660913,0],
                                           "starch_dusk (g/100gFW)":[1.3284677247171013,1.600011533626287,
                                                                     2.6254282537133005,1.0978029358571406],
                                           "hemicellulose_dusk (g/100gFW)":[1.089463383389083,
                                                                            0.6633103814170216,
                                                                            0.9510687588805857,
                                                                            1.0721624730031074],
                                           "simple sugars_dusk (g/100gFW)":[0.03202263184115184,
                                                                            0.35337416237039254,
                                                                            0.4138784209550903,
                                                                            0.3891937806184267],
                                           "sucrose_dusk (g/100gFW)":[0.46007757454985465,0.18700390989930638,
                                                                      0.3964371762172485,0.14206905936682546]})
df_matureleaf_content_1.set_index("Series",drop=True,inplace=True)
df_matureleaf_content_1["Δstarch"] = df_matureleaf_content_1["starch_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["starch_dawn (g/100gFW)"]
df_matureleaf_content_1["Δhemicellulose"] = df_matureleaf_content_1["hemicellulose_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["hemicellulose_dawn (g/100gFW)"]
df_matureleaf_content_1["Δsimple sugars"] = df_matureleaf_content_1["simple sugars_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["simple sugars_dawn (g/100gFW)"]
df_matureleaf_content_1["Δsucrose"] = df_matureleaf_content_1["sucrose_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["sucrose_dawn (g/100gFW)"]
df_matureleaf_content_1

,starch_dawn (g/100gFW),hemicellulose_dawn (g/100gFW),simple sugars_dawn (g/100gFW),sucrose_dawn (g/100gFW),starch_dusk (g/100gFW),hemicellulose_dusk (g/100gFW),simple sugars_dusk (g/100gFW),sucrose_dusk (g/100gFW),Δstarch,Δhemicellulose,Δsimple sugars,Δsucrose
Series,,,,,,,,,,,,
1,0.424568,1.089463,0.271941,0.271941,1.328468,1.089463,0.032023,0.460078,0.903900,0.000000,-0.239919,0.188136
2,0.497429,1.892274,0.650125,0.454127,1.600012,0.663310,0.353374,0.187004,1.102582,-1.228963,-0.296751,-0.267123
3,0.889538,0.861645,0.568619,0.097705,2.625428,0.951069,0.413878,0.396437,1.735890,0.089424,-0.154741,0.298732
4,1.252666,1.122133,0.553381,0.000000,1.097803,1.072162,0.389194,0.142069,-0.154863,-0.049971,-0.164187,0.142069


Series 3 ignored because of diurnal fluctuations in metabolite levels were not observed, calculate average from remaining data

In [8]:
df_matureleaf_content_1_std = pd.DataFrame(data={"avg. Δstarch (g/100gFW)":
                                                 [find_average(df_matureleaf_content_1["Δstarch"])],
                                                 "avg. Δstarch (%)":
                                                 [100*find_average(df_matureleaf_content_1["Δstarch"]/ \
                                                               df_matureleaf_content_1["starch_dusk (g/100gFW)"])],
                                                 "avg. Δsimple sugars (g/100gFW)":
                                                 [find_average(df_matureleaf_content_1["Δsimple sugars"])],
                                                 "avg. Δsucrose (g/100gFW)":
                                                 [find_average(df_matureleaf_content_1["Δsucrose"])],
                                                 "avg. Δsucrose (%)":
                                                  [100*find_average(df_matureleaf_content_1["Δsucrose"]/ \
                                                                df_matureleaf_content_1["sucrose_dusk (g/100gFW)"])]})
df_matureleaf_content_1_std

,avg. Δstarch (g/100gFW),avg. Δstarch (%),avg. Δsimple sugars (g/100gFW),avg. Δsucrose (g/100gFW),avg. Δsucrose (%)
0,0.896877,47.240852,-0.213899,0.090453,18.350658


In [9]:
print("Starch to sucrose ratio = "+str(float(df_matureleaf_content_1_std["avg. Δstarch (g/100gFW)"])/ \
                                             model.metabolites.GLC_c.formula_weight/ \
                                      (float(df_matureleaf_content_1_std["avg. Δsucrose (g/100gFW)"])/ \
                                             model.metabolites.SUCROSE_c.formula_weight)))

Starch to sucrose ratio = 18.839190878931536


### Leaf

Extract sucrose,hexose and starch content from Liu et al 2004, Figure 4 (PPFD = 600 umol/m2/s, photoperiod=12h, sampled between 11:00 and 13:00 h into photoperiod)

<img src="references/Screenshot_20190717_135552.png" height=50% width=50%>

In [10]:
df_leaf_content_1 = pd.DataFrame(data={"DAA":[0,3,5,7,10],
                                       "sucrose (mg/gDW)":[6.3492063492063275,5.0793650793650515,
                                                           5.0793650793650515,4.20634920634918,
                                                           1.746031746031723],
                                       "hexose (mg/gDW)":[61.64383561643821,81.36986301369862,
                                                          82.46575342465738,69.3150684931507,
                                                          45.5707762557078],
                                       "starch (mg/gDW)":[47.42268041237085,47.01030927835024,
                                                          37.525773195875956,35.25773195876279,
                                                          64.32989690721634]})
df_leaf_content_1.set_index("DAA",inplace=True,drop=True)
df_leaf_content_1

,sucrose (mg/gDW),hexose (mg/gDW),starch (mg/gDW)
DAA,,,
0,6.349206,61.643836,47.422680
3,5.079365,81.369863,47.010309
5,5.079365,82.465753,37.525773
7,4.206349,69.315068,35.257732
10,1.746032,45.570776,64.329897


Using Clements 1930 data on starch accumulated in 12h to estimate starch, hexose and sucrose content when diel fluctuations are ignored.

In [11]:
df_leaf_content_1_revised = pd.DataFrame(data={"sucrose (mg/gDW)":df_leaf_content_1["sucrose (mg/gDW)"]* \
                                                  (100-df_matureleaf_content_1_std["avg. Δsucrose (%)"][0])/100,
                                           "starch (mg/gDW)":df_leaf_content_1["starch (mg/gDW)"]* \
                                                  (100-df_matureleaf_content_1_std["avg. Δstarch (%)"][0])/100,
                                           "hexose (mg/gDW)":df_leaf_content_1["hexose (mg/gDW)"]})
df_leaf_content_1_revised

,sucrose (mg/gDW),starch (mg/gDW),hexose (mg/gDW)
DAA,,,
0,5.184085,25.019802,61.643836
3,4.147268,24.802239,81.369863
5,4.147268,19.798278,82.465753
7,3.434456,18.601679,69.315068
10,1.425623,33.939906,45.570776


Gather stach, hexose and sucrose content, 8hrs after illumination from Huber 2008, Table 1

In [12]:
df_leaf_content_2 = pd.DataFrame(data={"cultivar":["Maple Presto","Ransom"],
                                       "starch (mg/dm2)":[190,78],
                                       "hexose (mg/dm2)":[0,1.9],
                                       "sucrose (mg/dm2)":[8.7,4.9]})
df_leaf_content_2.set_index("cultivar",drop=True,inplace=True)
df_leaf_content_2

,starch (mg/dm2),hexose (mg/dm2),sucrose (mg/dm2)
cultivar,,,
Maple Presto,190,0.0,8.7
Ransom,78,1.9,4.9


Extract amino acid content from Chiozza et al 2010, Figure 2

<img src="references/Screenshot_20190717_143619.png" height=50% width=50%>

In [13]:
df_leaf_content_3 = pd.DataFrame(data={"Variety":["aphid resitant","aphid sensitive"],
                                       "GABA (nmol/50mgFW)":[0.8287671232876719,1.0136986301369868],
                                       "Asn (nmol/50mgFW)":[67.26027397260277,86.1643835616437],
                                       "Gln (nmol/50mgFW)":[82.87671232876716,99.31506849315056],
                                       "Glt (nmol/50mgFW)":[184.58904109589022,234.93150684931462],
                                       "His (nmol/50mgFW)":[11.778523489932848,13.657718120805297],
                                       "Pro (nmol/50mgFW)":[19.062499999999943,20.68181818181808],
                                       "Ser (nmol/50mgFW)":[87.28205128205093,100.4102564102559]})
df_leaf_content_3.set_index("Variety",drop=True,inplace=True)
df_leaf_content_3

,GABA (nmol/50mgFW),Asn (nmol/50mgFW),Gln (nmol/50mgFW),Glt (nmol/50mgFW),His (nmol/50mgFW),Pro (nmol/50mgFW),Ser (nmol/50mgFW)
Variety,,,,,,,
aphid resitant,0.828767,67.260274,82.876712,184.589041,11.778523,19.062500,87.282051
aphid sensitive,1.013699,86.164384,99.315068,234.931507,13.657718,20.681818,100.410256


Use Clements 1930 data to extract baseline sucrose, simple sugars, starch and hemicellulose data from diel metabolite levels

In [14]:
xlist = list()
ylist = list()
ylist2 = list()
ylist3 = list()
ylist4 = list()
for i in df_matureleaf_content_1.index:
    if i == 4:
        continue
    xlist.append(i)
    ylist.append(min(df_matureleaf_content_1["simple sugars_dawn (g/100gFW)"][i],
                     df_matureleaf_content_1["simple sugars_dusk (g/100gFW)"][i]))
    ylist2.append(min(df_matureleaf_content_1["sucrose_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["sucrose_dusk (g/100gFW)"][i]))
    ylist3.append(min(df_matureleaf_content_1["hemicellulose_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["hemicellulose_dusk (g/100gFW)"][i]))
    ylist4.append(min(df_matureleaf_content_1["starch_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["starch_dusk (g/100gFW)"][i]))
df_leaf_content_4 = pd.DataFrame(data={"Series":xlist,
                                       "baseline starch (g/100gFW)":ylist4,
                                       "baseline hemicellulose (g/100gFW)":ylist3,
                                       "baseline sucrose (g/100gFW)":ylist2,
                                       "baseline simple sugars (g/100gFW)":ylist})
df_leaf_content_4.set_index("Series",drop=True,inplace=True)
df_leaf_content_4

,baseline starch (g/100gFW),baseline hemicellulose (g/100gFW),baseline sucrose (g/100gFW),baseline simple sugars (g/100gFW)
Series,,,,
1,0.424568,1.089463,0.271941,0.032023
2,0.497429,0.663310,0.187004,0.353374
3,0.889538,0.861645,0.097705,0.413878


##### Data for soy sample analysis. Thanks to Yu and Pedro   #####
Starch data found here: Onedrive > FFAR > Soy and wheat biomass content > Soy > Starch.xlsx  
MW of D-(+)-Galactopyranose, pentakis(trimethylsilyl) ether (isomer 1) is [541.06](https://www.chemeo.com/cid/61-521-5/D-%28+%29-Galactopyranose%2C%20pentakis%28trimethylsilyl%29%20ether%20%28isomer%201%29)  
MW of D-Glucose, 2,3,4,5,6-pentakis-O-(trimethylsilyl)- is [541.06](https://www.chemeo.com/cid/61-521-5/D-%28+%29-Galactopyranose%2C%20pentakis%28trimethylsilyl%29%20ether%20%28isomer%201%29)  
MW of L(-)-Fucose, tetrakis(trimethylsilyl) ether is [452.88](https://www.chemeo.com/cid/61-521-5/D-%28+%29-Galactopyranose%2C%20pentakis%28trimethylsilyl%29%20ether%20%28isomer%201%29)  
  
*x* nmol/mL/gDW = *100x* nmol/gDW since 100mL was collected from 1gDW of sample. 

In [15]:

df_leaf_content_5_starch = pd.DataFrame(data={"sample":["L10","L11","L12","L13","L09","L14","L15","L16",
                                                                 "Blank01","Blank04","Blank06","Blank08","Blank09"],
                                                       "type":["ambient CO2","ambient CO2","ambient CO2","ambient CO2",
                                                               "elevated CO2","elevated CO2","elevated CO2","elevated CO2","-","-","-","-","-"],
                                                       "sample weight (mg)":[14.7,6.4,9.3,8.9,11.5,12.7,8.6,9.2,0,0,0,0,0],
                                                       "D-(+)-Galactopyranose, pentakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)":
                                                       [401.3341,41.4689,413.1693,312.3808,450.4626,4693.1360,594.7925,958.7851,104.1261,135.6329,105.4994,3225.1386,12612.9450],
                                                       "D-Glucose, 2,3,4,5,6-pentakis-O-(trimethylsilyl)- (ng/mL/mgDW)":
                                                       [6763.3455,7346.5070,6988.8045,4714.6736,4902.5363,6491.5553,6263.6940,9287.6320,373.3934,907.6334,1567.5501,3094.5004,3006.8918],
                                                       "L(-)-Fucose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)":
                                                       [5387.3067,7680.5241,6107.5979,6209.3119,5605.5879,6872.3415,9019.8503,8640.7707,222.9144,464.2251,875.2603,1102.1938,1139.1538]})

df_leaf_content_5_starch["Galactose (umol/mgDW)"]=0.1*df_leaf_content_5_starch["D-(+)-Galactopyranose, pentakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)"]/541.06
df_leaf_content_5_starch["Glucose (umol/mgDW)"]=0.1*df_leaf_content_5_starch["D-Glucose, 2,3,4,5,6-pentakis-O-(trimethylsilyl)- (ng/mL/mgDW)"]/541.06
df_leaf_content_5_starch["Fucose (umol/mgDW)"]=0.1*df_leaf_content_5_starch["L(-)-Fucose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)"]/452.88


df_leaf_content_5_starch

,sample,type,sample weight (mg),"D-(+)-Galactopyranose, pentakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)","D-Glucose, 2,3,4,5,6-pentakis-O-(trimethylsilyl)- (ng/mL/mgDW)","L(-)-Fucose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)",Galactose (umol/mgDW),Glucose (umol/mgDW),Fucose (umol/mgDW)
0,L10,ambient CO2,14.7,401.3341,6763.3455,5387.3067,0.074176,1.250018,1.189566
1,L11,ambient CO2,6.4,41.4689,7346.5070,7680.5241,0.007664,1.357799,1.695929
2,L12,ambient CO2,9.3,413.1693,6988.8045,6107.5979,0.076363,1.291688,1.348613
3,L13,ambient CO2,8.9,312.3808,4714.6736,6209.3119,0.057735,0.871377,1.371072
4,L09,elevated CO2,11.5,450.4626,4902.5363,5605.5879,0.083256,0.906098,1.237765
5,L14,elevated CO2,12.7,4693.1360,6491.5553,6872.3415,0.867397,1.199785,1.517475
6,L15,elevated CO2,8.6,594.7925,6263.6940,9019.8503,0.109931,1.157671,1.991665
7,L16,elevated CO2,9.2,958.7851,9287.6320,8640.7707,0.177205,1.716562,1.907960
8,Blank01,-,0.0,104.1261,373.3934,222.9144,0.019245,0.069011,0.049222
9,Blank04,-,0.0,135.6329,907.6334,464.2251,0.025068,0.167751,0.102505


In [53]:
glc = find_average(df_leaf_content_5_starch[df_leaf_content_5_starch["type"]=="ambient CO2"]["Glucose (umol/mgDW)"])
background = find_average(df_leaf_content_5_starch[df_leaf_content_5_starch["type"]=="-"]["Glucose (umol/mgDW)"])
print("Starch in GLC equivalents (umol/gDW):"+str(glc-background))

Starch in GLC equivalents (umol/gDW):0.8618894078290763


Cell wall data found here: Onedrive > FFAR > Soy and wheat biomass content > Soy > Cell_wall.xlsx  
MW of Mannose, 6-deoxy-2,3,4,5-tetrakis-O-(trimethylsilyl)-, L- is [452.88](https://www.chemeo.com/cid/71-391-9/Mannose%2C%206-deoxy-2%2C3%2C4%2C5-tetrakis-O-%28trimethylsilyl%29-%2C%20L-)  
MW of D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 1) is [438.85](https://www.chemeo.com/cid/40-349-0/D-Arabinopyranose%2C%20tetrakis%28trimethylsilyl%29%20ether%20%28isomer%201%29)  
MW of D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 2) is [438.85](https://www.chemeo.com/cid/33-373-1/D-Arabinopyranose%2C%20tetrakis%28trimethylsilyl%29%20ether%20%28isomer%202%29)  
MW of L-(+)-Rhamnopyranose, tetrakis(trimethylsilyl) ether is [452.88](https://www.chemeo.com/cid/26-135-3/L-%28+%29-Rhamnopyranose%2C%20tetrakis%28trimethylsilyl%29%20ether)  
MW of d-(+)-Xylose, tetrakis(trimethylsilyl) ether01 is [unknown](https://www.chemeo.com/search?q=d-%28%2B%29-Xylose%2C+tetrakis%28trimethylsilyl%29)  
MW of d-(+)-Xylose, tetrakis(trimethylsilyl) ether02 is [unknown](https://www.chemeo.com/search?q=d-%28%2B%29-Xylose%2C+tetrakis%28trimethylsilyl%29)  
MW of D-(+)-Galactopyranose, pentakis(trimethylsilyl) ether (isomer 1) is [541.06](https://www.chemeo.com/cid/61-521-5/D-%28+%29-Galactopyranose%2C%20pentakis%28trimethylsilyl%29%20ether%20%28isomer%201%29)  
MW of D-Glucose, 2,3,4,5,6-pentakis-O-(trimethylsilyl)- is [541.06](https://www.chemeo.com/cid/61-521-5/D-%28+%29-Galactopyranose%2C%20pentakis%28trimethylsilyl%29%20ether%20%28isomer%201%29)   
MW of L(-)-Fucose, tetrakis(trimethylsilyl) ether is [452.88](https://www.chemeo.com/cid/61-521-5/D-%28+%29-Galactopyranose%2C%20pentakis%28trimethylsilyl%29%20ether%20%28isomer%201%29)  
MW of Glucopyranose, pentakis-O-trimethylsilyl- is [541.1](https://pubchem.ncbi.nlm.nih.gov/compound/11038830)  
MW of d-(+)-Xylose, tetrakis(trimethylsilyl) ether is [unknown](https://www.chemeo.com/search?q=d-%28%2B%29-Xylose%2C+tetrakis%28trimethylsilyl%29)   

  
*x* nmol/mL/gDW = *100x* nmol/gDW since 100mL was collected from 1gDW of sample. 

In [59]:

df_leaf_content_5_cellwall = pd.DataFrame(data={"sample":["L10","L11","L12","L13","L09","L14","L15","L16",
                                                                 "Blank01","Blank04","Blank06","Blank08","Blank09"],
                                                       "type":["ambient CO2","ambient CO2","ambient CO2","ambient CO2",
                                                               "elevated CO2","elevated CO2","elevated CO2","elevated CO2","-","-","-","-","-"],
                                                       "sample weight (mg)":[14.7,6.4,9.3,8.9,11.5,12.7,8.6,9.2,0,0,0,0,0],
                                                       "Mannose, 6-deoxy-2,3,4,5-tetrakis-O-(trimethylsilyl)-, L- (ng/mL/mgDW)":
                                                       [853.0476,757.6553,1018.6382,1008.0476,846.6223,774.8793,785.3713,746.8259,4.6103,2.9960,6.6569,7.9003,2.0166],
                                                       "D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)":
                                                       [1489.3294,2224.3464,1870.4026,1963.8771,1511.6217,1311.0308,1785.3591,1901.8925,9.7439,7.3911,4.8005,6.4073,2.1784],
                                                       "D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 2) (ng/mL/mgDW)":
                                                       [1740.8721,2564.5670,2536.4653,2321.2585,1929.1629,1714.4810,2172.9359,2076.3582,107.9781,3338.1653,3286.9028,3272.2846,3250.2425],
                                                       "L-(+)-Rhamnopyranose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)":
                                                       [474.7554,395.0860,567.1393,509.2180,465.0697,445.0354,515.9588,345.7034,117.2403,1306.8548,1283.8851,1281.1625,1270.6035],
                                                       "d-(+)-Xylose, tetrakis(trimethylsilyl) ether01 (ng/mL/mgDW)":
                                                       [1280.2961,2162.1158,1817.1377,1950.8026,1570.7690,1253.0282,1708.8173,1653.1125,3287.6066,1431.4329,1411.1666,1404.0236,1392.6960],
                                                       "d-(+)-Xylose, tetrakis(trimethylsilyl) ether02 (ng/mL/mgDW)":
                                                       [768.7118,1409.6259,1161.4001,1194.3541,930.7999,813.8581,1121.0419,1042.9141,1408.9147,1425.3570,1402.1399,1394.7088,1383.4508],
                                                       "D-(+)-Galactopyranose, pentakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)":
                                                       [1227.5229,1850.6296,1839.5255,2181.5117,1967.6444,1801.6979,2213.5372,1794.8441,73.4667,17.3895,17.5042,15.8247,10.1389],
                                                       "D-Glucose, 2,3,4,5,6-pentakis-O-(trimethylsilyl)- (ng/mL/mgDW)":
                                                       [305.4558,390.8696,559.4491,641.9748,527.7318,698.6704,830.6719,462.6008,88.4206,53.4933,58.2416,63.5348,58.2891],
                                                       "L(-)-Fucose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)":
                                                       [248.9958,290.0291,375.8116,522.7878,365.6113,545.5028,532.4035,251.0517,32.3421,23.7589,31.0559,37.5947,11.8070],
                                                       "Glucopyranose, pentakis-O-trimethylsilyl- (ng/mL/mgDW)":
                                                       [91.4577,118.5490,167.2126,202.3010,170.6720,265.4535,359.0364,133.1297,20.4021,18.3046,18.9271,20.9563,34.1762],
                                                       "d-(+)-Xylose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)":
                                                       [1138.6524,1808.9924,1459.7559,1589.9850,1341.6589,1155.7426,1385.4971,1498.6046,2.680461346,3.011966102,2.599285061,6.198531368,1.784618053]
                                                       })


df_leaf_content_5_cellwall["Mannose (umol/mgDW)"]=0.1*df_leaf_content_5_cellwall["Mannose, 6-deoxy-2,3,4,5-tetrakis-O-(trimethylsilyl)-, L- (ng/mL/mgDW)"]/452.88
df_leaf_content_5_cellwall["Arabinose (umol/mgDW)"]=(0.1*df_leaf_content_5_cellwall["D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)"]/438.85)+(0.1*df_leaf_content_5_cellwall["D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 2) (ng/mL/mgDW)"]/541.06)
df_leaf_content_5_cellwall["Rhamnose (umol/mgDW)"]=0.1*df_leaf_content_5_cellwall["L-(+)-Rhamnopyranose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)"]/452.88
df_leaf_content_5_cellwall["Galactose (umol/mgDW)"]=0.1*df_leaf_content_5_cellwall["D-(+)-Galactopyranose, pentakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)"]/541.06
df_leaf_content_5_cellwall["Glucose (umol/mgDW)"]=(0.1*df_leaf_content_5_cellwall["D-Glucose, 2,3,4,5,6-pentakis-O-(trimethylsilyl)- (ng/mL/mgDW)"]/541.06)+(0.1*df_leaf_content_5_cellwall["Glucopyranose, pentakis-O-trimethylsilyl- (ng/mL/mgDW)"]/541.06)
df_leaf_content_5_cellwall["Fucose (umol/mgDW)"]=0.1*df_leaf_content_5_cellwall["L(-)-Fucose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)"]/452.88

df_leaf_content_5_cellwall

,sample,type,sample weight (mg),"Mannose, 6-deoxy-2,3,4,5-tetrakis-O-(trimethylsilyl)-, L- (ng/mL/mgDW)","D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)","D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 2) (ng/mL/mgDW)","L-(+)-Rhamnopyranose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)","d-(+)-Xylose, tetrakis(trimethylsilyl) ether01 (ng/mL/mgDW)","d-(+)-Xylose, tetrakis(trimethylsilyl) ether02 (ng/mL/mgDW)","D-(+)-Galactopyranose, pentakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)","D-Glucose, 2,3,4,5,6-pentakis-O-(trimethylsilyl)- (ng/mL/mgDW)","L(-)-Fucose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)","Glucopyranose, pentakis-O-trimethylsilyl- (ng/mL/mgDW)","d-(+)-Xylose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)",Mannose (umol/mgDW),Arabinose (umol/mgDW),Rhamnose (umol/mgDW),Galactose (umol/mgDW),Glucose (umol/mgDW),Fucose (umol/mgDW)
0,L10,ambient CO2,14.7,853.0476,1489.3294,1740.8721,474.7554,1280.2961,768.7118,1227.5229,305.4558,248.9958,91.4577,1138.652400,0.188361,0.661123,0.104830,0.226874,0.073358,0.054981
1,L11,ambient CO2,6.4,757.6553,2224.3464,2564.5670,395.0860,2162.1158,1409.6259,1850.6296,390.8696,290.0291,118.5490,1808.992400,0.167297,0.980847,0.087239,0.342038,0.094152,0.064041
2,L12,ambient CO2,9.3,1018.6382,1870.4026,2536.4653,567.1393,1817.1377,1161.4001,1839.5255,559.4491,375.8116,167.2126,1459.755900,0.224925,0.895001,0.125229,0.339985,0.134303,0.082983
3,L13,ambient CO2,8.9,1008.0476,1963.8771,2321.2585,509.2180,1950.8026,1194.3541,2181.5117,641.9748,522.7878,202.3010,1589.985000,0.222586,0.876526,0.112440,0.403192,0.156041,0.115436
4,L09,elevated CO2,11.5,846.6223,1511.6217,1929.1629,465.0697,1570.7690,930.7999,1967.6444,527.7318,365.6113,170.6720,1341.658900,0.186942,0.701003,0.102692,0.363665,0.129081,0.080730
5,L14,elevated CO2,12.7,774.8793,1311.0308,1714.4810,445.0354,1253.0282,813.8581,1801.6979,698.6704,545.5028,265.4535,1155.742600,0.171100,0.615617,0.098268,0.332994,0.178192,0.120452
6,L15,elevated CO2,8.6,785.3713,1785.3591,2172.9359,515.9588,1708.8173,1121.0419,2213.5372,830.6719,532.4035,359.0364,1385.497100,0.173417,0.808434,0.113928,0.409111,0.219885,0.117560
7,L16,elevated CO2,9.2,746.8259,1901.8925,2076.3582,345.7034,1653.1125,1042.9141,1794.8441,462.6008,251.0517,133.1297,1498.604600,0.164906,0.817138,0.076334,0.331727,0.110104,0.055434
8,Blank01,-,0.0,4.6103,9.7439,107.9781,117.2403,3287.6066,1408.9147,73.4667,88.4206,32.3421,20.4021,2.680461,0.001018,0.022177,0.025888,0.013578,0.020113,0.007141
9,Blank04,-,0.0,2.9960,7.3911,3338.1653,1306.8548,1431.4329,1425.3570,17.3895,53.4933,23.7589,18.3046,3.011966,0.000662,0.618652,0.288565,0.003214,0.013270,0.005246



### INCOMPLE###
soluble metabolites data found here: Onedrive > FFAR > Soy and wheat biomass content > Soy > Sol_Metab.xlsx  

  
*x* nmol/mL/gDW = *100x* nmol/gDW since 100mL was collected from 1gDW of sample. 

In [59]:
#### INCOMEPLETE###
df_leaf_content_5_cellwall = pd.DataFrame(data={"sample":["L10","L11","L12","L13","L09","L14","L15","L16",
                                                                 "Blank01","Blank04","Blank06","Blank08","Blank09"],
                                                       "type":["ambient CO2","ambient CO2","ambient CO2","ambient CO2",
                                                               "elevated CO2","elevated CO2","elevated CO2","elevated CO2","-","-","-","-","-"],
                                                       "sample weight (mg)":[14.7,6.4,9.3,8.9,11.5,12.7,8.6,9.2,0,0,0,0,0],
                                                       "Mannose, 6-deoxy-2,3,4,5-tetrakis-O-(trimethylsilyl)-, L- (ng/mL/mgDW)":
                                                       [853.0476,757.6553,1018.6382,1008.0476,846.6223,774.8793,785.3713,746.8259,4.6103,2.9960,6.6569,7.9003,2.0166],
                                                       "D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)":
                                                       [1489.3294,2224.3464,1870.4026,1963.8771,1511.6217,1311.0308,1785.3591,1901.8925,9.7439,7.3911,4.8005,6.4073,2.1784],
                                                       "D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 2) (ng/mL/mgDW)":
                                                       [1740.8721,2564.5670,2536.4653,2321.2585,1929.1629,1714.4810,2172.9359,2076.3582,107.9781,3338.1653,3286.9028,3272.2846,3250.2425],
                                                       "L-(+)-Rhamnopyranose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)":
                                                       [474.7554,395.0860,567.1393,509.2180,465.0697,445.0354,515.9588,345.7034,117.2403,1306.8548,1283.8851,1281.1625,1270.6035],
                                                       "d-(+)-Xylose, tetrakis(trimethylsilyl) ether01 (ng/mL/mgDW)":
                                                       [1280.2961,2162.1158,1817.1377,1950.8026,1570.7690,1253.0282,1708.8173,1653.1125,3287.6066,1431.4329,1411.1666,1404.0236,1392.6960],
                                                       "d-(+)-Xylose, tetrakis(trimethylsilyl) ether02 (ng/mL/mgDW)":
                                                       [768.7118,1409.6259,1161.4001,1194.3541,930.7999,813.8581,1121.0419,1042.9141,1408.9147,1425.3570,1402.1399,1394.7088,1383.4508],
                                                       "D-(+)-Galactopyranose, pentakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)":
                                                       [1227.5229,1850.6296,1839.5255,2181.5117,1967.6444,1801.6979,2213.5372,1794.8441,73.4667,17.3895,17.5042,15.8247,10.1389],
                                                       "D-Glucose, 2,3,4,5,6-pentakis-O-(trimethylsilyl)- (ng/mL/mgDW)":
                                                       [305.4558,390.8696,559.4491,641.9748,527.7318,698.6704,830.6719,462.6008,88.4206,53.4933,58.2416,63.5348,58.2891],
                                                       "L(-)-Fucose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)":
                                                       [248.9958,290.0291,375.8116,522.7878,365.6113,545.5028,532.4035,251.0517,32.3421,23.7589,31.0559,37.5947,11.8070],
                                                       "Glucopyranose, pentakis-O-trimethylsilyl- (ng/mL/mgDW)":
                                                       [91.4577,118.5490,167.2126,202.3010,170.6720,265.4535,359.0364,133.1297,20.4021,18.3046,18.9271,20.9563,34.1762],
                                                       "d-(+)-Xylose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)":
                                                       [1138.6524,1808.9924,1459.7559,1589.9850,1341.6589,1155.7426,1385.4971,1498.6046,2.680461346,3.011966102,2.599285061,6.198531368,1.784618053]
                                                       })


df_leaf_content_5_cellwall["Mannose (umol/mgDW)"]=0.1*df_leaf_content_5_cellwall["Mannose, 6-deoxy-2,3,4,5-tetrakis-O-(trimethylsilyl)-, L- (ng/mL/mgDW)"]/452.88
df_leaf_content_5_cellwall["Arabinose (umol/mgDW)"]=(0.1*df_leaf_content_5_cellwall["D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)"]/438.85)+(0.1*df_leaf_content_5_cellwall["D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 2) (ng/mL/mgDW)"]/541.06)
df_leaf_content_5_cellwall["Rhamnose (umol/mgDW)"]=0.1*df_leaf_content_5_cellwall["L-(+)-Rhamnopyranose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)"]/452.88
df_leaf_content_5_cellwall["Galactose (umol/mgDW)"]=0.1*df_leaf_content_5_cellwall["D-(+)-Galactopyranose, pentakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)"]/541.06
df_leaf_content_5_cellwall["Glucose (umol/mgDW)"]=(0.1*df_leaf_content_5_cellwall["D-Glucose, 2,3,4,5,6-pentakis-O-(trimethylsilyl)- (ng/mL/mgDW)"]/541.06)+(0.1*df_leaf_content_5_cellwall["Glucopyranose, pentakis-O-trimethylsilyl- (ng/mL/mgDW)"]/541.06)
df_leaf_content_5_cellwall["Fucose (umol/mgDW)"]=0.1*df_leaf_content_5_cellwall["L(-)-Fucose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)"]/452.88

df_leaf_content_5_cellwall

,sample,type,sample weight (mg),"Mannose, 6-deoxy-2,3,4,5-tetrakis-O-(trimethylsilyl)-, L- (ng/mL/mgDW)","D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)","D-Arabinopyranose, tetrakis(trimethylsilyl) ether (isomer 2) (ng/mL/mgDW)","L-(+)-Rhamnopyranose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)","d-(+)-Xylose, tetrakis(trimethylsilyl) ether01 (ng/mL/mgDW)","d-(+)-Xylose, tetrakis(trimethylsilyl) ether02 (ng/mL/mgDW)","D-(+)-Galactopyranose, pentakis(trimethylsilyl) ether (isomer 1) (ng/mL/mgDW)","D-Glucose, 2,3,4,5,6-pentakis-O-(trimethylsilyl)- (ng/mL/mgDW)","L(-)-Fucose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)","Glucopyranose, pentakis-O-trimethylsilyl- (ng/mL/mgDW)","d-(+)-Xylose, tetrakis(trimethylsilyl) ether (ng/mL/mgDW)",Mannose (umol/mgDW),Arabinose (umol/mgDW),Rhamnose (umol/mgDW),Galactose (umol/mgDW),Glucose (umol/mgDW),Fucose (umol/mgDW)
0,L10,ambient CO2,14.7,853.0476,1489.3294,1740.8721,474.7554,1280.2961,768.7118,1227.5229,305.4558,248.9958,91.4577,1138.652400,0.188361,0.661123,0.104830,0.226874,0.073358,0.054981
1,L11,ambient CO2,6.4,757.6553,2224.3464,2564.5670,395.0860,2162.1158,1409.6259,1850.6296,390.8696,290.0291,118.5490,1808.992400,0.167297,0.980847,0.087239,0.342038,0.094152,0.064041
2,L12,ambient CO2,9.3,1018.6382,1870.4026,2536.4653,567.1393,1817.1377,1161.4001,1839.5255,559.4491,375.8116,167.2126,1459.755900,0.224925,0.895001,0.125229,0.339985,0.134303,0.082983
3,L13,ambient CO2,8.9,1008.0476,1963.8771,2321.2585,509.2180,1950.8026,1194.3541,2181.5117,641.9748,522.7878,202.3010,1589.985000,0.222586,0.876526,0.112440,0.403192,0.156041,0.115436
4,L09,elevated CO2,11.5,846.6223,1511.6217,1929.1629,465.0697,1570.7690,930.7999,1967.6444,527.7318,365.6113,170.6720,1341.658900,0.186942,0.701003,0.102692,0.363665,0.129081,0.080730
5,L14,elevated CO2,12.7,774.8793,1311.0308,1714.4810,445.0354,1253.0282,813.8581,1801.6979,698.6704,545.5028,265.4535,1155.742600,0.171100,0.615617,0.098268,0.332994,0.178192,0.120452
6,L15,elevated CO2,8.6,785.3713,1785.3591,2172.9359,515.9588,1708.8173,1121.0419,2213.5372,830.6719,532.4035,359.0364,1385.497100,0.173417,0.808434,0.113928,0.409111,0.219885,0.117560
7,L16,elevated CO2,9.2,746.8259,1901.8925,2076.3582,345.7034,1653.1125,1042.9141,1794.8441,462.6008,251.0517,133.1297,1498.604600,0.164906,0.817138,0.076334,0.331727,0.110104,0.055434
8,Blank01,-,0.0,4.6103,9.7439,107.9781,117.2403,3287.6066,1408.9147,73.4667,88.4206,32.3421,20.4021,2.680461,0.001018,0.022177,0.025888,0.013578,0.020113,0.007141
9,Blank04,-,0.0,2.9960,7.3911,3338.1653,1306.8548,1431.4329,1425.3570,17.3895,53.4933,23.7589,18.3046,3.011966,0.000662,0.618652,0.288565,0.003214,0.013270,0.005246


### Summary

- Since amino acid content was only available in g/gFW and FW/DW data was not available, diel starch and sucrose levels (in g/gFW) were used to predict baseline starch and sucrose levels in leaf


In [17]:
biomass.at["sSUCROSE_b","leaf"]=1000*find_average(df_leaf_content_4["baseline sucrose (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["Starch_b","leaf"]=1000*find_average(df_leaf_content_4["baseline starch (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("GLC_c").formula_weight
biomass.at["GLC_c","leaf"]=1000*find_average(df_leaf_content_4["baseline simple sugars (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("GLC_c").formula_weight

biomass.at["ASN_c","leaf"]= 100*0.001*find_average(df_leaf_content_3["Asn (nmol/50mgFW)"])/50
biomass.at["sGABA_b","leaf"]= 100*0.001*find_average(df_leaf_content_3["GABA (nmol/50mgFW)"])/50
biomass.at["sGLN_b","leaf"]= 100*0.001*find_average(df_leaf_content_3["Gln (nmol/50mgFW)"])/50
biomass.at["sGLU_b","leaf"]= 100*0.001*find_average(df_leaf_content_3["Glt (nmol/50mgFW)"])/50
biomass.at["HIS_c","leaf"]= 100*0.001*find_average(df_leaf_content_3["His (nmol/50mgFW)"])/50
biomass.at["PRO_c","leaf"]= 100*0.001*find_average(df_leaf_content_3["Pro (nmol/50mgFW)"])/50
biomass.at["sSER_b","leaf"]= 100*0.001*find_average(df_leaf_content_3["Ser (nmol/50mgFW)"])/50

In [18]:
biomass

,leaf,stem,root,seed
,,,,
sSUCROSE_b,0.542075,0.0,0.0,0.0
GLC_c,1.478859,0.0,0.0,0.0
FRU_c,0.000000,0.0,0.0,0.0
Starch_b,3.351791,0.0,0.0,0.0
Cellulose_b,0.000000,0.0,0.0,0.0
Xylan_b,0.000000,0.0,0.0,0.0
L_PHOSPHATIDATE_p,0.000000,0.0,0.0,0.0
PHOSPHATIDYL_CHOLINE_r,0.000000,0.0,0.0,0.0
L_1_PHOSPHATIDYL_ETHANOLAMINE_r,0.000000,0.0,0.0,0.0


### Stem



Extract diurnal starch, sucrose, hemicellulose and simple sugar content from Clements 1930, Figure 6, 9, 13 and 14. Based on Figure 1 and 2, dawn = 8am and dusk = 8pm

<img src="references/Screenshot_20190717_161155.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160304.png" height=50% width=50%>

<img src="references/Screenshot_20190717_161259.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160242.png" height=50% width=50%>

In [18]:
df_stem_content = pd.DataFrame(data={"Series":[1,2,3,4],
                                     "simple sugars_dawn (g/100gFW)":[0.08928305478569998,1.0069230769230781,
                                                                      0.22941176470588265,0.755800478743633],
                                     "sucrose_dawn (g/100gFW)":[0.24999412110523256,0.10918376068376556,
                                                                0.14117647058823524,0.36680114242250283],
                                     "simple sugars_dusk (g/100gFW)":[0.27423868312757205,0.9930341880341897,
                                                                      0.23368616214394122,0.8292719666520014],
                                     "sucrose_dusk (g/100gFW)":[0,0.08606410256411046,
                                                                0.1748626327321765,0.2553567833200341]})
df_stem_content.set_index("Series",drop=True,inplace=True)
df_stem_content["avg. Δsimple sugars (g/100gFW)"] = df_stem_content["simple sugars_dusk (g/100gFW)"]- \
                                                        df_stem_content["simple sugars_dawn (g/100gFW)"]
df_stem_content["avg. Δsucrose (g/100gFW)"] = df_stem_content["sucrose_dusk (g/100gFW)"]- \
                                                        df_stem_content["sucrose_dawn (g/100gFW)"]

xlist = list()
ylist = list()
ylist2 = list()
for i in df_matureleaf_content_1.index:
    xlist.append(i)
    ylist.append(min(df_stem_content["simple sugars_dawn (g/100gFW)"][i],
                     df_stem_content["simple sugars_dusk (g/100gFW)"][i]))
    ylist2.append(min(df_stem_content["sucrose_dawn (g/100gFW)"][i],
                      df_stem_content["sucrose_dusk (g/100gFW)"][i]))
df_stem_content_std = pd.DataFrame(data={"Series":xlist,
                                       "baseline sucrose (g/100gFW)":ylist2,
                                       "baseline simple sugars (g/100gFW)":ylist})
df_stem_content_std.set_index("Series",drop=True,inplace=True)
df_stem_content_std

,baseline simple sugars (g/100gFW),baseline sucrose (g/100gFW)
Series,,
1,0.089283,0.000000
2,0.993034,0.086064
3,0.229412,0.141176
4,0.755800,0.255357


### Summary

- Sucrose, hexose content data is per FW and conversion to DW is not available

Assumptions

In [19]:
biomass.at["sSUCROSE_b","stem"] = 1000*find_average(df_stem_content_std["baseline sucrose (g/100gFW)"])/ \
                                        model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["GLC_c","stem"]= 1000*find_average(df_stem_content_std["baseline simple sugars (g/100gFW)"])/ \
                                        model.metabolites.get_by_id("GLC_c").formula_weight

### Root

Gather root exudate content from Canarini et al 2016, Figure 2

<img src="references/Screenshot_20190718_145459.png" height=50% width=50%>

In [20]:
print("amount of C exudate ="+str(8.689956331877875)+" ug/gDWroot/2h")
print("amount of N exudate ="+str(1.315789473684226)+" ug/gDWroot/2h")

df_root_exudate_1 = pd.DataFrame(data={"%C or %N":["%C","%N"],"Sugars":[14.947368421052634,0],
                                       "Amino acids":[25.684210526315788 - 14.947368421052634,
                                                      67.89473684210535 - 25.789473684210833],
                                       "Organic acids":[30.105263157894733 - 25.684210526315788,0],
                                       "Inorganic ions":[0,67.89473684210535 - 25.789473684210833]})
df_root_exudate_1.set_index("%C or %N",inplace=True,drop=True)
df_root_exudate_1

amount of C exudate =8.68995633188 ug/gDWroot/2h
amount of N exudate =1.31578947368 ug/gDWroot/2h


,Amino acids,Inorganic ions,Organic acids,Sugars
%C or %N,,,,
%C,10.736842,0.000000,4.421053,14.947368
%N,42.105263,42.105263,0.000000,0.000000


Gathering fatty-acid content from Thomas et al 2007 Table I

In [21]:
df_root_content_1 = pd.DataFrame(data={"Tissue type":["Conrad - Epidermis","OX760-6 - Epidermis",
                                                      "Conrad - Endodermis","OX760-6 - Endodermis"],
                                       "Fatty acid 16:1":[0.21,0.05,0.82,0.39],
                                       "Fatty acid 16:0":[0.39,0.25,1.50,0.72],
                                       "Fatty acid 18:1":[0.71,0.18,7.46,1.46],
                                       "Fatty acid 18:0":[0.32,0.04,0.83,0.30],
                                       "Fatty acid 20:0":[0.05,0.03,0.28,0.49],
                                       "Fatty acid 22:0":[0.09,0.04,0.31,0.31],
                                       "Total (ug/cm2)":[1.77,0.59,11.20,3.67]})
df_root_content_1.set_index("Tissue type",inplace=True,drop=True)
df_root_content_1

,Fatty acid 16:0,Fatty acid 16:1,Fatty acid 18:0,Fatty acid 18:1,Fatty acid 20:0,Fatty acid 22:0,Total (ug/cm2)
Tissue type,,,,,,,
Conrad - Epidermis,0.39,0.21,0.32,0.71,0.05,0.09,1.77
OX760-6 - Epidermis,0.25,0.05,0.04,0.18,0.03,0.04,0.59
Conrad - Endodermis,1.50,0.82,0.83,7.46,0.28,0.31,11.20
OX760-6 - Endodermis,0.72,0.39,0.30,1.46,0.49,0.31,3.67


In [22]:
biomass.at["PALMITATE_c","root"] = find_average(df_root_content_1["Fatty acid 16:0"])
biomass.at["CPD_9245_p","root"] = find_average(df_root_content_1["Fatty acid 16:1"])
biomass.at["STEARIC_ACID_p","root"] = find_average(df_root_content_1["Fatty acid 18:0"])
biomass.at["OLEATE_CPD_p","root"] = find_average(df_root_content_1["Fatty acid 18:1"])
biomass.at["ARACHIDIC_ACID_p","root"] = find_average(df_root_content_1["Fatty acid 20:0"])
biomass.at["DOCOSANOATE_p","root"] = find_average(df_root_content_1["Fatty acid 22:0"])

### Summary

- sugars, amino acids and organic acids as %C and %N data could not be used directly.
- lipid composition in ug/cm2 cannot be translated to mg/gDW

### Seed


Gather non-reducing sugars, reducing sugars, starch, proteins, phospholipids, glycolipids, sterols and free fatty acid content from Guleria et al 2008, Table 1

In [23]:
df_seed_content = pd.DataFrame(data={"seed type":["large","small"],
                               "starch (mg/gDW)":[49.7,42.3],
                               "reducing sugars (mg/gDW)":[24.9,27.7],
                               "total soluble sugars (mg/gDW)":[125,101],
                               "proteins (mg/gDW)":[377,264],
                               "lipid content (mg/gDW)":[241,224],
                               "phospholipids (mg/gDW lipid)":[57.1,58.7],
                               "glycolipids (mg/gDW lipid)":[10.6,10.3],
                               "sterols (mg/gDW lipid)":[26.8,26.4],
                               "free fatty acids (mg/gDW lipid)":[0.1,0.1],
                               "16:0 (mg/gDW fatty acds)":[132,122],
                               "18:0 (mg/gDW fatty acds)":[29,38],
                               "18:1 (mg/gDW fatty acds)":[269,245],
                               "18:2 (mg/gDW fatty acds)":[513,518],
                               "18:3 (mg/gDW fatty acds)":[56,76]})
df_seed_content_std = pd.DataFrame(data={"seed type":["large","small"],
                               "starch (mg/gDW)":[49.7,42.3],
                               "reducing sugars (mg/gDW)":[24.9,27.7],
                               "non-reducing sugars (mg/gDW)":df_seed_content["total soluble sugars (mg/gDW)"]-\
                                         df_seed_content["reducing sugars (mg/gDW)"],
                               "proteins (mg/gDW)":[377,264],
                               "phospholipids (mg/gDW)":df_seed_content["phospholipids (mg/gDW lipid)"]* \
                                   df_seed_content["lipid content (mg/gDW)"],
                               "glycolipids (mg/gDW)":df_seed_content["glycolipids (mg/gDW lipid)"]* \
                                   df_seed_content["lipid content (mg/gDW)"],
                               "sterols (mg/gDW)":df_seed_content["sterols (mg/gDW lipid)"]* \
                                   df_seed_content["lipid content (mg/gDW)"],
                               "16:0 (mg/gDW)":df_seed_content["16:0 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:0 (mg/gDW)":df_seed_content["18:0 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:1 (mg/gDW)":df_seed_content["18:1 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:2 (mg/gDW)":df_seed_content["18:2 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:3 (mg/gDW)":df_seed_content["18:3 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"]})
df_seed_content_std.set_index("seed type",inplace=True,drop=True)
df_seed_content_std

,16:0 (mg/gDW),18:0 (mg/gDW),18:1 (mg/gDW),18:2 (mg/gDW),18:3 (mg/gDW),glycolipids (mg/gDW),non-reducing sugars (mg/gDW),phospholipids (mg/gDW),proteins (mg/gDW),reducing sugars (mg/gDW),starch (mg/gDW),sterols (mg/gDW)
seed type,,,,,,,,,,,,
large,3181.2,698.9,6482.9,12363.3,1349.6,2554.6,100.1,13761.1,377,24.9,49.7,6458.8
small,2732.8,851.2,5488.0,11603.2,1702.4,2307.2,73.3,13148.8,264,27.7,42.3,5913.6


Extract FW/DW, lipid, protein, starch, soluble sugar and reducing sugar content from Sharma et al 2013, Figure 1 and 2


<img src="references/Screenshot_20190719_101524.png" height=25% width=25%>

<img src="references/Screenshot_20190719_101729.png" height=25% width=25%>

In [24]:
df_seed_content_2 = pd.DataFrame(data={"node position":range(4,18),
                               "DW/FW":[0.928872804935928,0.927787138111058,0.934610821072615,
                                        0.938872804935928,0.937322021831989,0.918329378262933,
                                        0.95515068818225,0.944299952539155,0.945538680588514,
                                        0.948406502135738,0.948249881347888,0.948559563360228,
                                        0.956778595158994,0.958483626008543],
                               "lipid content (mg/gDW)":[242.2236872277,264.320110066498,255.514790185737,
                                                         247.835932125659,237.345218986471,237.531529465719,
                                                         235.470648933731,225.550332492547,212.250630589314,
                                                         211.313345562944,207.564205457464,207.750515936712,
                                                         216.922724145838,221.603416647558],
                               "proteins (mg/gDW)":[251.291824869483,241.120372567632,245.770052206929,
                                                    222.503856193641,236.746559088752,239.656502135738,
                                                    269.587980541054,271.621381110584,278.882890365449,
                                                    334.986058376839,327.426435690555,323.350735643094,
                                                    316.663205980066,321.312885619364],
                               "starch (mg/gDW)":[42.5131154156578,40.2199354317999,48.283393866021,
                                                  37.240213882163,41.1970338983051,45.3324253430186,
                                                  42.681598062954,40.3894269572236,39.7033898305085,
                                                  49.195419693301,40.4736682808717,42.8233454398709,
                                                  39.8153753026634,35.5579096045198],
                               "total soluble sugars (mg/gDW)":[90.6887971663593,88.8516144140979,
                                                                88.4657943268639,105.697140013466,
                                                                81.2716255378942,91.6635929861538,
                                                                98.7394982582477,91.7227247445918,
                                                                86.7779046280847,86.8071777758262,
                                                                97.1985597611311,90.5968794824508,
                                                                81.5069816457364,90.6548403149791],
                               "reducing sugars (mg/gDW)":[112.999279159929,151.966724463752,
                                                           122.654639677375,137.206647314383,
                                                           155.484034366537,157.207036957665,
                                                           115.897056245007,65.4822809717704,
                                                           49.4141713261512,53.6188119776346,
                                                           58.6545617487192,78.173352295973,
                                                           46.7922616844281,52.2406436907015]})
df_seed_content_2.set_index("node position",drop=True,inplace=True)
df_seed_content_2

,DW/FW,lipid content (mg/gDW),proteins (mg/gDW),reducing sugars (mg/gDW),starch (mg/gDW),total soluble sugars (mg/gDW)
node position,,,,,,
4,0.928873,242.223687,251.291825,112.999279,42.513115,90.688797
5,0.927787,264.320110,241.120373,151.966724,40.219935,88.851614
6,0.934611,255.514790,245.770052,122.654640,48.283394,88.465794
7,0.938873,247.835932,222.503856,137.206647,37.240214,105.697140
8,0.937322,237.345219,236.746559,155.484034,41.197034,81.271626
9,0.918329,237.531529,239.656502,157.207037,45.332425,91.663593
10,0.955151,235.470649,269.587981,115.897056,42.681598,98.739498
11,0.944300,225.550332,271.621381,65.482281,40.389427,91.722725
12,0.945539,212.250631,278.882890,49.414171,39.703390,86.777905


### Summary

Assumptions

- Sharma et al 2013 data has reducing  sugar content higher than total soluble sugar content and so given that the same data was available from Guleria et al 2008, this data was ignored

In [25]:
biomass.at["sSUCROSE_b","seed"] = find_average(df_seed_content_std["non-reducing sugars (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["GLC_c","seed"]=find_average(df_seed_content_std["reducing sugars (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("GLC_c").formula_weight
biomass.at["PALMITATE_c","seed"]=find_average(df_seed_content_std["16:0 (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("PALMITATE_c").formula_weight
biomass.at["STEARIC_ACID_p","seed"]=find_average(df_seed_content_std["18:0 (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("PALMITATE_c").formula_weight
biomass.at["OLEATE_CPD_p","seed"]=find_average(df_seed_content_std["18:1 (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("PALMITATE_c").formula_weight
biomass.at["Octadecadienoate_p","seed"]=find_average(df_seed_content_std["18:2 (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("PALMITATE_c").formula_weight
biomass.at["LINOLENIC_ACID_p","seed"]=find_average(df_seed_content_std["18:3 (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("PALMITATE_c").formula_weight
biomass.at["L_PHOSPHATIDATE_p","seed"]=find_average(df_seed_content_std["phospholipids (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("L_PHOSPHATIDATE_p").formula_weight
biomass.at["Starch_b","seed"]=find_average(df_seed_content_std["starch (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("GLC_c").formula_weight
biomass.at["Protein_b","seed"]=find_average(df_seed_content_std["proteins (mg/gDW)"])/proteinMW
biomass.to_csv("Data/biomass_soy.csv")
biomass

,leaf,root,seed,stem
,,,,
sSUCROSE_b,0.542075,0.0000,0.253289,0.352470
GLC_c,0.778346,0.0000,0.145985,2.869084
FRU_c,0.000000,0.0000,0.000000,0.000000
Starch_b,1.764099,0.0000,0.255334,0.000000
Cellulose_b,0.000000,0.0000,0.000000,0.000000
Xylan_b,0.000000,0.0000,0.000000,0.000000
L_PHOSPHATIDATE_p,0.000000,0.0000,20.799909,0.000000
PHOSPHATIDYL_CHOLINE_r,0.000000,0.0000,0.000000,0.000000
L_1_PHOSPHATIDYL_ETHANOLAMINE_r,0.000000,0.0000,0.000000,0.000000


### References

- Liu, F., Jensen, C.R. and Andersen, M.N. (2004) Drought stress effect on carbohydrate concentration in soybean leaves and pods during early reproductive development: its implication in altering pod set. F. Crop. Res., 86, 1–13. https://doi.org/10.1016/S0378-4290(03)00165-5.
- Chiozza, M. V, O’Neal, M.E. and MacIntosh, G.C. (2010) Constitutive and Induced Differential Accumulation of Amino Acid in Leaves of Susceptible and Resistant Soybean Plants in Response to the Soybean Aphid (Hemiptera: Aphididae). Environ. Entomol., 39, 856–864. https://doi.org/10.1603/EN09338.
- Huber, S.C. (2008) Biochemical Mechanism for Regulation of Sucrose Accumulation in Leaves during Photosynthesis. PLANT Physiol., 91, 656–662. https://doi.org/10.1104/pp.91.2.656.
- White, M.C., Decker, A.M. and Chaney, R.L. (2008) Metal Complexation in Xylem Fluid : I. CHEMICAL COMPOSITION OF TOMATO AND SOYBEAN STEM EXUDATE. PLANT Physiol., 67, 292–300. https://doi.org/10.1104/pp.67.2.292.
- Clements, H.F. (1930) Hourly Variations in Carbohydrate Content of Leaves and Petioles. Bot. Gaz., 89, 241–272. https://www.jstor.org/stable/2471041.
- Canarini, A., Merchant, A. and Dijkstra, F.A. (2016) Drought effects on Helianthus annuus and Glycine max metabolites: from phloem to root exudates. Rhizosphere, 2, 85–97. https://doi.org/10.1016/j.rhisph.2016.06.003.
- Thomas, R., Fang, X., Ranathunge, K., Anderson, T.R., Peterson, C.A. and Bernards, M.A. (2007) Soybean Root Suberin: Anatomical Distribution, Chemical Composition, and Relationship to Partial Resistance to Phytophthora sojae. PLANT Physiol., 144, 299–311. https:doi.org/10.1104/pp.106.091090.
- Guleria, S., Sharma, S., Gill, B.S. and Munshi, S.K. (2008) Distribution and biochemical composition of large and small seeds of soybean (Glycine max L.). J. Sci. Food Agric., 88, 269–272. https://doi.org/10.1002/jsfa.2825.
- Sharma, S., Kaur, A., Bansal, A. and Gill, B.S. (2013) Positional effects on soybean seed composition during storage. J. Food Sci. Technol., 50, 353–359. https://doi.org/10.1007/s13197-011-0341-0.